---
### Minería de Datos (Máster en Ciencia de Datos)
# Práctica: k-NN
### Rodrigo Manzanas, Ana Casanueva  
#### Departamento de Matemática Aplicada y Ciencias de la Computación (Universidad de Cantabria)
---

Como se explicó en la sesión de teoría, la técnica k-NN puede utilizarse tanto para clasificación como para regresión. En esta práctica vamos a trabajar con el dataset *MNIST* para clasificación de dígitos en imágenes y con datos meteorológicos para predecir la lluvia diaria en Lisboa. Utilizaremos los paquetes `class`, `FNN`y `caret`.

## k-NN para clasificación
Para este ejemplo utilizaremos el dataset *MNIST*. Como ya habéis visto, se trata de reconocer dígitos (0, 1, ..., 9) en una colección de imágenes. En primer lugar cargamos el dataset (sólo la parte de train) con la función *read.csv* y convertimos a factor nuestra variable objetivo (primera columna).

In [ ]:
## loading dataset
data = read.csv("/home/rodrigo/work/DOCENCIA/2024-2025/M1966_mineria_datos_data_science/S10_kNN/data/MNIST_train.csv")

## converting target variable to factor
data[,1] = as.factor(data[,1])
#str(data)
#head(data, 10)

Para entender mejor qué aspecto tienen las imágenes que contiene el dataset, vamos a dibujar un par de ellas (las que tú quieras) en un *p-color*. Utiliza para ello las funciones `image.plot` (paquete `fields`) y `fliplr` (paquete `pracma`).

In [ ]:
library(fields)
library(pracma)

# p-colors
par(mfrow = c(1, 2))
image.plot(1:28, 1:28, fliplr(matrix(as.numeric(data[30, -1]), 
                                  nrow = 28, ncol = 28, byrow = F)), 
        xlab = "pixel", ylab = "pixel")

image.plot(1:28, 1:28, fliplr(matrix(as.numeric(data[50, -1]), 
                                  nrow = 28, ncol = 28, byrow = F)), 
        xlab = "pixel", ylab = "pixel")

Puesto que el dataset es muy grande (casi 4000 muestras), trabajaremos sólo con las 2000 primeras instancias. De entre éstas, nuestro dataset de train estará formado por las 1500 primeras (75% del total de datos), y el de test por las 500 restantes (25% del total de datos).

In [20]:
## train/test partition
indtrain = 1:1500
indtest = 1501:2000
data.train = data[indtrain,] 
data.test = data[indtest,]

**Ejercicio:** Calcula, en la muestra de train, cuántas observaciones tienes para cada dígito. ¿Dírias que hay desbalanceo?

In [ ]:
n.digit = sapply(0:9, function(dig) sum(data.train[, 1] == dig))
bp = barplot(n.digit, ylim = c(0, 200), xlab = "digit", ylab = "no. occurreces")
axis(1, at = bp, labels = 0:9)
grid(nx = NA, ny = NULL)

**Ejercicio:** Utiliza la función `knn` (paquete `class`) para clasificar los dígitos del test, considerando para ello el vecino más cecano en el train. Calcula la tasa de aciertos (total) sobre el test.

In [ ]:
## k-NN (with k=1)
library(class)
pred = knn(data.train[,-1], data.test[,-1], data.train$label, k = 1)
sum(pred == data.test$label) / length(pred)

**Ejercicio:** Como acabas de ver, el error "global" de clasificación del método k-NN (con k=1) está en torno al 10%. Dibuja un barplot con la tasa de aciertos para cada dígito (0, 1, ..., 9) que nos permita hacer un ranking con los mejor y peor clasificados. ¿Cuál es el dígito que mejor se clasifica? ¿Y el peor? 

In [ ]:
## best and worst classified digits
acc.digit = rep(NA, 10)
for (digit in 0:9) {
  ind.digit = which(data.test$label == digit) 
  acc.digit[digit + 1] = sum(pred[ind.digit] == digit) / length(ind.digit)
}
bp = barplot(acc.digit, ylim = c(0, 1), xlab = "digit", ylab = "ACC")
axis(1, at = bp, labels = 0:9)
grid(nx = NA, ny = NULL)

**Ejercicio:** Obtén la matriz de confusión (función `confusionMatrix` del paquete `caret`). ¿Se confirman tus conclusiones? ¿A qué crees que pueden deberse?

In [ ]:
library(caret)
confusionMatrix(pred, as.factor(data.test$label))

**Ejercicio:** Para tratar de entender un poco mejor estos resultados, visualiza los nueve primeros "1" y los nueve primeros "8" que aparecen en el dataset. 

In [ ]:
## nine first "1" images
ind1 = which(data[,1] == 1)
par(mfrow = c(3,3))
for (i in 1:9) {
  image.plot(1:28, 1:28, fliplr(matrix(as.numeric(data[ind1[i], -1]), 
                                  nrow = 28, ncol = 28, byrow = F)), 
        xlab = "pixel", ylab = "pixel")
}

In [ ]:
## nine first "8" images
ind8 = which(data[,1] == 8)
par(mfrow = c(3,3))
for (i in 1:9) {
  image.plot(1:28, 1:28, fliplr(matrix(as.numeric(data[ind8[i], -1]), 
                                  nrow = 28, ncol = 28, byrow = F)), 
        xlab = "pixel", ylab = "pixel")
}

**Ejercicio:** Para evaluar cómo influye la elección del parámetro k en nuestros resultados, repite el mismo experimento pero considerando todos los *k* entre 1 y 20. ¿Se obtiene alguna conclusión general sobre el efecto que tiene la elección de *k*? ¿Es ese efecto más importante para algún dígito concreto?

In [ ]:
## effect of changing k
n.k = 1:20
acc.kd = matrix(NA, length(n.k), 10)
for (k in n.k) {
  print(paste("... predicting for", k, "nearest neighbors ...") )
  pred = knn(data.train[,-1], data.test[,-1], data.train$label, k = k)
  
  acc.digit = rep(NA, 10)
  for (digit in 0:9) {
    ind.digit = which(data.test$label == digit) 
    acc.digit[digit+1] = sum(pred[ind.digit] == digit) / length(ind.digit)
  }
  acc.kd[which(n.k == k), ] = acc.digit
}
## boxplot
boxplot(acc.kd, type = "l", lty = 1, 
        xlab = "digit", ylab = "accuracy", 
        names = 0:9)
grid()

## lineplot
matplot(acc.kd, type = "l", lty = 1, col = rainbow(10),
        xlab = "k", ylab = "acc")
legend("topright", as.character(0:9), 
       col = rainbow(10), lty = 1)
grid()

## k-NN para regresión 
Como ya se explicó, en el *downscaling* estadístico se trata de predecir variables meteorológicas a escala local (por ejemplo la precipitación y/o temperatura observada en una localidad concreta) a partir de variables de larga escala dadas por un modelo númerico del clima cuya resolución espacial es grosera (por ejemplo, la presión o los vientos en dominios sinópticos). En este ejemplo utilizaremos la técnica k-NN para intentar predecir la lluvia diaria observada en Lisboa (predictando) a partir de un conjunto de variables de larga escala (predictores).  
En primer lugar cargamos el dataset *meteo.csv*. Como el dataset es muy grande nos quedaremos sólo con los primeros 2000 días (filas). Además, eliminaremos la primera columna (es un simple contador). A partir de este momento tendremos un dataset (llámalo *df*) con 2000 filas y 321 columnas (compruébalo). La primera columna será nuestra variable objetivo: la precipitación diaria en Lisboa durante el período 1979-2008. Las 320 columnas restantes serán nuestros predictores: 8 variables de larga escala, definidas sobre una malla de 40 puntos (8 en la longitud y 5 en la latitud) que cubre la Península Ibérica.

- Altura geopotencial en 500 hPa (columnas 2:41)
- Temperatura del aire en 850 hPa (columnas 42:81), 700 hPa (columnas 82:121) y 500 hPa (columnas 122:161)
- Temperatura del aire en superficie (columnas 162:201)
- Humedad específica del aire en 850 hPa (columnas 202:241) y 500 hPa (columnas 242:281)
- Presión al nivel del mar (columnas 282:321)

In [9]:
## loading data
data = read.csv("/home/rodrigo/work/DOCENCIA/2024-2025/M1966_mineria_datos_data_science/S10_kNN/data/meteo.csv")
df = data[1:2000, -1]; rm(data)

Por comodidad, renombramos como *precip* nuestra variable objetivo. 

In [ ]:
colnames(df)[1] = "precip"

Además, para entender un poco mejor qué tipo de datos contiene el dataset, vamos a dibujar en un mapa la temperatura del aire en 850 hPa (columnas 42:81) y la presión al nivel del mar (columnas 282:321) para el primer día de registros.

In [ ]:
lon = seq(-10, 4, 2) # 2º resolution
lat = seq(36, 44, 2) # 2º resolution

library(maps)
par(mfrow = c(1, 2))
image.plot(x = lon, y = lat, matrix(as.numeric(df[1, 42:81]), 
                                     nrow = length(lon), 
                                     ncol = length(lat), byrow = T),
           main = "T850")
map(add = T)  # overlapping map

image.plot(x = lon, y = lat, matrix(as.numeric(df[1, 282:321]), 
                                    nrow = length(lon), 
                                    ncol = length(lat), byrow = T),
           main = "SLP")
map(add = T)  # overlapping map

Ya que las vamos a necesitar posteriormente, crearemos una nueva variable *y* (vector predictando con la precipitación en Lisboa) y otra *x* (matriz con los predictores de larga escala).

In [10]:
## new variables x and y
y = df[, 1]  # predictand (vector)
x = df[, -1]  # predictors (matrix)

**Ejercicio:** Para visualizar los datos, dibuja en una figura la serie diaria de precipitación observada y en otra el valor promedio de cada uno de los 320 predictores.
¿Qué dirías sobre los predictores? ¿Crees que habría que tener algún cuidado especial al trabajar con la técnica k-NN en este caso?

In [ ]:
## data visualization
par(mfrow = c(2, 1))
plot(y, xlab = "days", ylab = "precip (mm)", type = "l"); grid()
plot(colMeans(x), cex = 0.1, xlab = "predictors", ylab = "units"); grid()

Nuestro siguiente paso será dividir el dataset total en dos subconjuntos independientes de train y test (75% y 25%, respectivamente), escogidos aleatoriamente. Por comodidad, crea las variables *df.train*, *df.test*, *x.train*, *y.train*, *x.test*, *y.test*.

In [12]:
## train/test partition
n = nrow(df)
indtrain = sample(1:n, round(0.75*n))
indtest = setdiff(1:n, indtrain)

# 75% train
df.train = df[indtrain, ]
x.train = x[indtrain, ]
y.train = y[indtrain]

# 25% test
df.test = df[indtest, ]
x.test = x[indtest, ]
y.test = y[indtest]

Ya estamos en condiciones de buscar el *k* óptimo para la técnica k-NN. Para ello emplearemos el paquete `caret` (método `knn`).   

**Ejercicio:** Trabaremos ahora con `caret`. En concreto, considera un marco de validación cruzada con 4 _folds_ sobre el dataset de train y barre todos los *k* desde 1 a 30 (revisa la documentación sobre el argumento `tuneGrid` de la función `train`). Recuerda que habrá que estandarizar los predictores (argumento `preProcess` de `train`). ¿Cómo varía la métrica de validación RMSE con *k*? ¿Por qué crees que ocurre esto? ¿Cuál es el *k* óptimo?   
**Nota:** Puedes consultar la documentación del paquete `caret` aquí: https://topepo.github.io/caret

In [ ]:
## fitting the value of k with caret
trctrl = trainControl(method = "cv", number = 4)  #4-fold CV
knn.fit = train(precip ~ ., df.train,
                method = "knn",
                trControl = trctrl,
                preProcess = c("center", "scale"),
                tuneGrid = expand.grid(k = 1:30))
plot(knn.fit)
knn.fit$bestTune

Hemos visto que el RMSE disminuye con *k* (hasta un cierto punto). Pero para determinar cuán buena/mala es nuestra predicción no podemos fijarnos en una única medida, sino que debemos tener en cuenta un abanico más amplio de métricas que nos permitan caracterizar otros aspectos de la predicción que puedan ser de interés. En esta práctica consideraremos, además del RMSE, las siguientes métricas de validación:  

* Tasa de aciertos (o *accuracy*): permite evaluar el evento binario *lluvia/no lluvia*. Se suele tomar la cantidad 0.1 mm como umbral para definir días de lluvia.
* Correlación de Spearman: permite evaluar cuán bien la serie temporal predicha (completa) sigue la observación. Se puede calcular en `R` utilizando la función `cor`.
* *Ratio* de varianzas: permite evaluar hasta qué punto la variabilidad total de nuestra predicción (serie completa) se asemeja a la observada. Se calcula como $\frac{var(pred)}{var(obs}$

**Ejercicio:** Utiliza el *k* óptimo que acabas de encontrar para predecir la lluvia en el test, y valida los resultados en función de estas 4 métricas. ¿Qué resultados obtienes? ¿Dirías que tu predicción es buena? ¿Mala? ¿Por qué?

In [ ]:
## predictions for test
pred = predict(knn.fit, df.test[, -1])

## validation
# RMSE
rmse <- function(x, y) {
    stopifnot(length(x) == length(y))
    sqrt(mean((x - y)^2))
}
val.rmse = rmse(df.test$precip, pred)

# Spearman correlation
val.r = cor(df.test$precip, pred, method = "spearman")

## accuracy binary
acc.class = function(x, y) {
  stopifnot(length(x) == length(y))
  return(sum(diag(table(x, y))) / length(x))
}
val.bin = acc.class(ifelse(df.test$precip < 0.1, 0, 1), ifelse(pred < 0.1, 0, 1))

# ratio of variances
val.rv = var(pred) / var(df.test$precip)

cbind(val.rmse, val.r, val.bin, val.rv)

**Ejercicio:** A continuación, vamos a analizar cómo varían las 4 métricas de validación consideradas con *k*, pero esta vez apoyándonos en la función `knn.reg` (paquete `FNN`). Para ello, crea un bucle que barra todos los *k* desde 1 hasta 30 y calcula en cada iteración el RMSE, la correlación de Spearman, el accuracy (binario) y el ratio de varianzas. Haz este experimento sin escalar los predictores y escalándolos correctamente (usa para ello las funciones `preProcess` y `scale`). Grafica los resultados. ¿Qué conclusiones obtienes? ¿Cuál sería para tí un *k* óptimo?   
**Nota:** Utiliza la función `knn.reg` (paquete `FNN`).

In [ ]:
library(FNN)

## scaling the predictors
params.scaling = preProcess(x.train, method = c("center", "scale"))
x.train.scaled = scale(x.train, 
                       center = params.scaling$mean, scale = params.scaling$std)
x.test.scaled = scale(x.test, 
                      center = params.scaling$mean, scale = params.scaling$std)

val.rmse.without.scaling = rep(NA, 30)
val.r.without.scaling = rep(NA, 30)
val.bin.without.scaling = rep(NA, 30)
val.rv.without.scaling = rep(NA, 30)

val.rmse.with.scaling = rep(NA, 30)
val.r.with.scaling = rep(NA, 30)
val.bin.with.scaling = rep(NA, 30)
val.rv.with.scaling = rep(NA, 30)
for (k in 1:30) {
  print(sprintf("... trying k = %d ...", k))
  
  ## predicting
  pred.without.scaling = knn.reg(train = x.train, test = x.test, 
                                 y = y.train, k = k)
  pred.with.scaling = knn.reg(train = x.train.scaled, test = x.test.scaled, 
                              y = y.train, k = k)
  
  ## validating (without predictor scaling)
  val.rmse.without.scaling[k] = rmse(y.test, pred.without.scaling$pred)
  val.r.without.scaling[k] = cor(y.test, pred.without.scaling$pred, method = "spearman")
  val.bin.without.scaling[k] = acc.class(ifelse(y.test < 0.1, 0, 1), 
                                         ifelse(pred.without.scaling$pred < 0.1, 0, 1))
  val.rv.without.scaling[k] = var(pred.without.scaling$pred) / var(y.test)
  
  ## validating (with predictor scaling)
  val.rmse.with.scaling[k] = rmse(y.test, pred.with.scaling$pred)
  val.r.with.scaling[k] = cor(y.test, pred.with.scaling$pred, method = "spearman")
  val.bin.with.scaling[k] = acc.class(ifelse(y.test < 0.1, 0, 1), 
                                      ifelse(pred.with.scaling$pred < 0.1, 0, 1))
  val.rv.with.scaling[k] = var(pred.with.scaling$pred) / var(y.test)
}

## plotting
par(mfrow = c(2,2))
matplot(1:30, cbind(val.rmse.without.scaling, val.rmse.with.scaling),
        type = "o", pch = 19, cex = 0.5, xlab = "k", ylab = "RMSE")
legend("topright", c("Without scaling", "With scaling"), 
       col = c("black", "red"), lty = 1); grid()
matplot(1:30, cbind(val.r.without.scaling, val.r.with.scaling),
        type = "o", pch = 19, cex = 0.5, xlab = "k", ylab = "Spearman corr.")
legend("topright", c("Without scaling", "With scaling"), 
       col = c("black", "red"), lty = 1); grid()
matplot(1:30, cbind(val.bin.without.scaling, val.bin.with.scaling), 
        type = "o", pch = 19, cex = 0.5, xlab = "k", ylab = "Acc. binary")
legend("topright", c("Without scaling", "With scaling"), 
       col = c("black", "red"), lty = 1); grid()
matplot(1:30, cbind(val.rv.without.scaling, val.rv.with.scaling),
        type = "o", pch = 19, cex = 0.5, xlab = "k", ylab = "RV")
legend("topright", c("Without scaling", "With scaling"), 
       col = c("black", "red"), lty = 1); grid()

En la sesión de teoría se comentó que es frecuente el uso de técnicas avanzadas (como el análisis de Componentes Principales) que permiten reducir la dimensionalidad del problema sin pérdida de información efectiva. Estas técnicas se verán en detalle en la asignatura _Estadística para la Ciencia de Datos_. De momento, para reducir el número de predictores que entran en nuestro modelo, nos limitaremos a efectuar un entresacado (no informado) de variables. En concreto, escogeremos una de cada cinco.

In [16]:
## not informed selection
nx = ncol(x.train)
ind.extr = seq(1, nx, 5)

Otra forma un poco más dirigida de reducir la dimensionalidad de nuestro problema consiste en realizar un análisis de correlaciones. 

**Ejercicio:** Calcula la correlación de Spearman entre nuestra variable objetivo (lluvia) y todas las variables predictoras (larga escala) disponibles. La idea es que, cuanto más fuerte sea esta correlación, mayor es el vínculo físico entre predictando y predictor, y por tanto, más útil es la información que nos aporta ese predictor. Este análisis nos permite descartar aquellos predictores cuya correlación con el predictando no supere cierto umbral (nótese, sin embargo, que no se resolvería el problema de la colinealidad en los predictores, que puede afectar negativamente a otro tipo de técnicas como los modelos lineales). Siguiendo esta idea, analiza la correlación existente entre nuestro predictando y los 320 predictores disponibles, y elimina aquellos que muestren valores entre -0.4 y 0.4 ¿Cuánto se ha reducido la dimensionalidad del problema?

In [ ]:
## informed selection
r.xy = c()
for (ivar in 2:nx) {
  r.xy[ivar] = cor(y.train, x.train[, ivar], method = "spearman")
}
plot(r.xy, ylim = c(-0.6, 0.6), pch = 19, cex = 0.5, 
     xlab = "predictor", ylab = "Spearman corr. with precip")
grid()

ind.sele = which(abs(r.xy) > 0.4)
length(ind.sele)
points(ind.sele, r.xy[ind.sele], col = "red", cex = 0.5)

**Ejercicio:** Obtén ahora la predicción en el test para el caso del entresacado no informado de predictores y para la selección informada de los mismos (recuerda escalar correctamente), para valores de *k* desde 1 hasta 30. Apóyate para ello en la función `knn.reg`. Evalúa estas predicciones en función de las 4 métricas de validación consideradas en el análisis anterior. Plotea en el mismo gráfico los resultados obtenidos para 1) todos los predictores, 2) el entresacado no informado y 3) la selección informada. ¿Qué conclusiones obtienes?

In [ ]:
## not informed selection
x.train.extr = x.train[, ind.extr]
x.test.extr = x.test[, ind.extr]
params.scaling.extr = preProcess(x.train.extr, method = c("center", "scale"))
x.train.extr.scaled = scale(x.train.extr,
                            center = params.scaling.extr$mean, 
                            scale = params.scaling.extr$std)
x.test.extr.scaled = scale(x.test.extr, 
                           center = params.scaling.extr$mean, 
                           scale = params.scaling.extr$std)

## informed selection
x.train.sele = x.train[, ind.sele]
x.test.sele = x.test[, ind.sele]
params.scaling.sele = preProcess(x.train.sele, method = c("center", "scale"))
x.train.sele.scaled = scale(x.train.sele,
                            center = params.scaling.sele$mean, 
                            scale = params.scaling.sele$std)
x.test.sele.scaled = scale(x.test.sele, 
                           center = params.scaling.sele$mean, 
                           scale = params.scaling.sele$std)


val.extr.rmse = rep(NA, 30)
val.extr.r = rep(NA, 30)
val.extr.bin = rep(NA, 30)
val.extr.rv = rep(NA, 30)

val.sele.rmse = rep(NA, 30)
val.sele.r = rep(NA, 30)
val.sele.bin = rep(NA, 30)
val.sele.rv = rep(NA, 30)

for (k in 1:30) {
  print(sprintf("... trying k = %d ...", k))
  
  ## not informed selection
  pred.extr = knn.reg(train = x.train.extr.scaled, test = x.test.extr.scaled, 
                      y = y.train, k = k)
  val.extr.rmse[k] = rmse(y.test, pred.extr$pred)
  val.extr.r[k] = cor(y.test, pred.extr$pred, method = "spearman")
  val.extr.bin[k] = acc.class(ifelse(y.test < 0.1, 0, 1), 
                              ifelse(pred.extr$pred < 0.1, 0, 1))
  val.extr.rv[k] = var(pred.extr$pred) / var(y.test)
  
  ## informed selection
  pred.sele = knn.reg(train = x.train.sele.scaled, test = x.test.sele.scaled,
                      y = y.train, k = k)
  val.sele.rmse[k] = rmse(y.test, pred.sele$pred)
  val.sele.r[k] = cor(y.test, pred.sele$pred, method = "spearman")
  val.sele.bin[k] = acc.class(ifelse(y.test < 0.1, 0, 1), 
                              ifelse(pred.sele$pred < 0.1, 0, 1))
  val.sele.rv[k] = var(pred.sele$pred) / var(y.test)    
}

## plotting
par(mfrow = c(2,2))
matplot(1:30, cbind(val.rmse.with.scaling, val.extr.rmse, val.sele.rmse), lty = 1, type = "o", pch = 19, cex = 0.5, col = c("black", "red", "green"), xlab = "k", ylab = "RMSE"); grid()
matplot(1:30, cbind(val.r.with.scaling, val.extr.r, val.sele.r), lty = 1, type = "o", pch = 19, cex = 0.5, col = c("black", "red", "green"), xlab = "k", ylab = "Spearman corr."); grid()
matplot(1:30, cbind(val.bin.with.scaling, val.extr.bin, val.sele.bin), lty = 1, type = "o", pch = 19, cex = 0.5, col = c("black", "red", "green"), xlab = "k", ylab = "Acc. binary"); grid()
matplot(1:30, cbind(val.rv.with.scaling, val.extr.rv, val.sele.rv), lty = 1, type = "o", pch = 19, cex = 0.5, col = c("black", "red", "green"), xlab = "k", ylab = "RV"); grid()
legend("topright", c("all", "not informed", "informed"), lty = 1, col = c("black", "red", "green"))